# All Events by District

districtCode=FMA

In [2]:
import requests
import json
import os

apikey = os.environ['FRC_API_KEY']

url = "https://frc-api.firstinspires.org/v3.0/2023/events?districtCode=FMA"

payload={}
headers = {
  'Authorization': 'Basic '+apikey,
  'If-Modified-Since': ''
}

response = requests.request("GET", url, headers=headers, data=payload)
events = response.json()

print(*(event['code'] for event in events['Events']))


MRCMP NJFLA NJROB NJSKI NJTAB NJWAS PABEN PAHAT PAPHI


# Teams by Event

Using eventCode=

In [3]:
url = "https://frc-api.firstinspires.org/v3.0/2023/teams?districtCode=FMA&eventCode=PAPHI"

payload={}
headers = {
  'Authorization': 'Basic '+apikey,
  'If-Modified-Since': ''
}

response = requests.request("GET", url, headers=headers, data=payload)
json = response.json()
print(json["teams"][0]["teamNumber"])

102


## Determine Teams per Event

In [4]:
import requests
import json

eventCodes = [event['code'] for event in events['Events']]
  
base_eventCode_url = "https://frc-api.firstinspires.org/v3.0/2023/teams?districtCode=FMA&eventCode="

event_urls = {}
for eventCode in eventCodes:
  event_urls[eventCode] = base_eventCode_url+eventCode

payload={}
headers = {
  'Authorization': 'Basic '+apikey,
  'If-Modified-Since': ''
}

teams_at_events = {}

for eventCode, url in event_urls.items():
  response = requests.request("GET", url, headers=headers, data=payload)
  if response.status_code == 200:
    event_teams = response.json()
    for team in event_teams["teams"]:
      if eventCode in teams_at_events:
        teams_at_events[eventCode].append(team["teamNumber"])
      else:
        teams_at_events[eventCode] = [team["teamNumber"]]
  else:
    print(f"mlff: get.py: {eventCode} {response.status_code}")


for event, teams in teams_at_events.items():
  print(f"{event}: {teams}")


NJFLA: [11, 41, 56, 75, 102, 193, 219, 222, 223, 484, 555, 752, 1143, 1279, 1672, 1676, 1811, 1923, 1989, 2070, 2458, 2577, 3142, 3515, 4361, 4573, 5438, 5732, 5992, 6860, 6945, 7045, 8117, 8513, 8628, 9116]
NJROB: [87, 293, 303, 316, 714, 1089, 1257, 1403, 1626, 1807, 1812, 2016, 2191, 2495, 2554, 2590, 3314, 4652, 4653, 5624, 5666, 5684, 6897, 7587, 8075, 8130, 8704, 8707, 8714, 9015, 9116]
NJSKI: [25, 41, 75, 204, 293, 752, 1089, 1257, 1279, 1403, 1626, 1672, 1923, 2070, 2495, 2554, 2577, 3340, 3515, 3637, 4361, 4750, 5438, 5992, 6016, 6860, 6897, 7587, 8075, 8513, 8588, 8706, 8714]
NJTAB: [25, 87, 203, 204, 272, 365, 816, 1168, 1647, 1712, 1807, 1812, 2180, 2191, 2559, 2720, 2722, 4342, 4750, 5113, 5457, 5732, 5895, 6226, 6921, 7110, 8130, 8630, 8704, 8721, 9100]
NJWAS: [11, 56, 193, 219, 222, 223, 303, 555, 714, 834, 1143, 1672, 1676, 1811, 1989, 2070, 3142, 3340, 4285, 4361, 4573, 4652, 6943, 6945, 7045, 8117, 8588, 8628, 8706, 8707, 9015]
PABEN: [272, 316, 321, 484, 486, 1391, 1

# Collect Rankings

Rankings for 2019, 2021, 2022

In [5]:
import statistics
url23 = "https://frc-api.firstinspires.org/v3.0/2022/rankings/district?districtCode=FMA"
url22 = "https://frc-api.firstinspires.org/v3.0/2022/rankings/district?districtCode=FMA"
url19 = "https://frc-api.firstinspires.org/v3.0/2019/rankings/district?districtCode=FMA"
url18 = "https://frc-api.firstinspires.org/v3.0/2018/rankings/district?districtCode=MAR"

rank_year_urls = {2018:url18, 2019:url19, 2022:url22, 2023:url23}

payload={}
headers = {
  'Authorization': 'Basic '+apikey,
  'If-Modified-Since': ''
}

#teams_and_ranks_by_year = {2018:{}, 2019:{}, 2022:{}}
teams_with_ranks_each_year = {}

for year, url in rank_year_urls.items():
  response = requests.request("GET", url, headers=headers, data=payload)
  if response.status_code == 200:
    team_ranks_for_year = response.json()
    for ranks_for_team in team_ranks_for_year["districtRanks"]:
      team_num = ranks_for_team["teamNumber"]
      if ranks_for_team["event2Points"] is None:
        continue
      if team_num in teams_with_ranks_each_year:
        if year in teams_with_ranks_each_year[team_num]:
          teams_with_ranks_each_year[team_num][year].append(ranks_for_team["event1Points"])
          teams_with_ranks_each_year[team_num][year].append(ranks_for_team["event2Points"])
        else:
          teams_with_ranks_each_year[team_num][year] = [ranks_for_team["event1Points"]]
          teams_with_ranks_each_year[team_num][year].append(ranks_for_team["event2Points"])
      else:
          teams_with_ranks_each_year[team_num] = {}
          if year in teams_with_ranks_each_year[team_num]:
            teams_with_ranks_each_year[year] = [ranks_for_team["event1Points"]]
            teams_with_ranks_each_year[team_num][year].append(ranks_for_team["event2Points"])
          else:
            teams_with_ranks_each_year[team_num][year] = [ranks_for_team["event1Points"]]
            teams_with_ranks_each_year[team_num][year].append(ranks_for_team["event2Points"])
  else:
    print(f"mlff: get.py: {year} {response.status_code}")

#print(json.dumps(ranks_2019, indent=2))
print(teams_with_ranks_each_year[2590])

{2018: [63.0, 73.0], 2019: [49.0, 71.0], 2022: [66.0, 71.0], 2023: [66.0, 71.0]}


## Sort Teams per Event by Rank

Using statistical mean of `event1Points` and `event2Points` for each team

In [6]:
def average_of_all_year_pts(tuple_for_team: tuple):
    sum = 0
    i = 0
    team_number, dict_of_data = tuple_for_team
    for year, list_of_points in dict_of_data.items():
        sum += statistics.mean(list_of_points)
    i += 1
    return sum/i
    
    
unsorted_teams = {team: data for team, data in teams_with_ranks_each_year.items() if 2023 in teams_with_ranks_each_year[team]}

sorted_teams = sorted(unsorted_teams.items(), reverse=True, key=average_of_all_year_pts)
print(sorted_teams)

[(2590, {2018: [63.0, 73.0], 2019: [49.0, 71.0], 2022: [66.0, 71.0], 2023: [66.0, 71.0]}), (1640, {2018: [49.0, 60.0], 2019: [63.0, 62.0], 2022: [61.0, 73.0], 2023: [61.0, 73.0]}), (5895, {2018: [24.0, 43.0], 2019: [33.0, 45.0], 2022: [73.0, 73.0], 2023: [73.0, 73.0]}), (1923, {2018: [50.0, 67.0], 2019: [47.0, 33.0], 2022: [53.0, 64.0], 2023: [53.0, 64.0]}), (2539, {2018: [29.0, 23.0], 2019: [63.0, 48.0], 2022: [69.0, 61.0], 2023: [69.0, 61.0]}), (834, {2018: [53.0, 68.0], 2019: [48.0, 71.0], 2022: [44.0, 42.0], 2023: [44.0, 42.0]}), (1807, {2018: [22.0, 31.0], 2019: [58.0, 60.0], 2022: [59.0, 57.0], 2023: [59.0, 57.0]}), (4342, {2018: [50.0, 45.0], 2019: [71.0, 34.0], 2022: [49.0, 48.0], 2023: [49.0, 48.0]}), (25, {2018: [72.0, 66.0], 2019: [43.0, 42.0], 2022: [55.0, 28.0], 2023: [55.0, 28.0]}), (1218, {2018: [52.0, 56.0], 2019: [33.0, 54.0], 2022: [41.0, 56.0], 2023: [41.0, 56.0]}), (222, {2018: [53.0, 53.0], 2019: [66.0, 33.0], 2022: [57.0, 29.0], 2023: [57.0, 29.0]}), (1403, {2018:

In [9]:
for event, teams in teams_at_events.items():  
    print(f"{event}: ", end=' ')
    for team, _ in sorted_teams:
        if team in teams:
            print(team, end=' ')
    print()


NJFLA: 
1923 222 56 1676 41 75 11 3142 223 102 2577 219 1672 1811 5732 7045 484 
NJROB: 
2590 1807 1403 3314 2016 1257 316 4653 1626 1089 2191 293 2495 7587 
NJSKI: 
1923 25 1403 41 75 1257 3637 1626 1089 2577 293 2495 7587 1672 
NJTAB: 
5895 1807 4342 25 365 272 6226 7110 2180 203 2191 1168 5732 
NJWAS: 
834 222 56 1676 11 3142 223 219 4285 1672 1811 7045 
PABEN: 
3314 2016 272 1391 2607 5401 316 7414 4285 4373 486 5490 484 
PAHAT: 
2590 1640 5895 2539 1807 25 1218 341 365 103 708 433 272 1391 5407 2607 5181 3637 4653 6226 7110 2180 7414 2095 5490 2234 
PAPHI: 
1640 2539 834 4342 1218 341 103 708 433 5407 5181 5401 102 203 1168 4373 2095 486 2234 
